In [20]:
import matplotlib.image as mpimg
import numpy as np
import cv2

In [2]:
# Read in the image
image = mpimg.imread('cutout1.jpg')

# Take histograms in R, G, and B
rhist = np.histogram(image[:,:,0], bins=32, range=(0, 256))
ghist = np.histogram(image[:,:,1], bins=32, range=(0, 256))
bhist = np.histogram(image[:,:,2], bins=32, range=(0, 256))

In [3]:
rhist[0]

array([400, 441, 302, 244, 189, 283, 412, 433, 239, 177, 209, 302, 276,
       179, 129, 106, 104, 111, 106,  96, 111, 123, 139, 130, 167, 164,
       149, 156, 222, 268, 258,  53])

In [6]:
rhist[1]

array([   0.,    8.,   16.,   24.,   32.,   40.,   48.,   56.,   64.,
         72.,   80.,   88.,   96.,  104.,  112.,  120.,  128.,  136.,
        144.,  152.,  160.,  168.,  176.,  184.,  192.,  200.,  208.,
        216.,  224.,  232.,  240.,  248.,  256.])

In [8]:
bin_edges = rhist[1]

In [9]:
bin_edges[1:]

array([   8.,   16.,   24.,   32.,   40.,   48.,   56.,   64.,   72.,
         80.,   88.,   96.,  104.,  112.,  120.,  128.,  136.,  144.,
        152.,  160.,  168.,  176.,  184.,  192.,  200.,  208.,  216.,
        224.,  232.,  240.,  248.,  256.])

In [12]:
bin_edges[0:len(bin_edges)-1]

array([   0.,    8.,   16.,   24.,   32.,   40.,   48.,   56.,   64.,
         72.,   80.,   88.,   96.,  104.,  112.,  120.,  128.,  136.,
        144.,  152.,  160.,  168.,  176.,  184.,  192.,  200.,  208.,
        216.,  224.,  232.,  240.,  248.])

In [13]:
bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
bin_centers

array([   4.,   12.,   20.,   28.,   36.,   44.,   52.,   60.,   68.,
         76.,   84.,   92.,  100.,  108.,  116.,  124.,  132.,  140.,
        148.,  156.,  164.,  172.,  180.,  188.,  196.,  204.,  212.,
        220.,  228.,  236.,  244.,  252.])

In [14]:
np.concatenate((rhist[0], ghist[0], bhist[0]))

array([400, 441, 302, 244, 189, 283, 412, 433, 239, 177, 209, 302, 276,
       179, 129, 106, 104, 111, 106,  96, 111, 123, 139, 130, 167, 164,
       149, 156, 222, 268, 258,  53,  95, 219, 441, 331, 235, 179, 188,
       324, 340, 321, 252, 215, 242, 296, 244, 198, 133, 117,  98, 127,
        88, 120, 104,  88, 117, 172, 152, 142, 133, 159, 219, 589, 106,
       206, 367, 337, 230, 174, 235, 328, 316, 238, 189, 227, 261, 271,
       291, 244, 179, 148, 134, 112, 129, 123,  94,  93, 160, 138, 117,
       110, 121, 148, 219, 633])

In [15]:
image.shape

(63, 106, 3)

In [16]:
max(image.shape[0], image.shape[1], 64) / 64

1.65625

In [17]:
106 / 64

1.65625

In [21]:
scale = max(image.shape[0], image.shape[1], 64) / 64
img_small = cv2.resize(image, (np.int(image.shape[1] / scale), np.int(image.shape[0] / scale)), interpolation=cv2.INTER_NEAREST)

In [22]:
img_small.shape

(38, 64, 3)

In [23]:
image = mpimg.imread('test_img.jpg')
small_img = cv2.resize(image, (32, 32))
print(small_img.shape)

(32, 32, 3)


In [24]:
small_img.ravel()

array([86, 71, 52, ..., 78, 73, 80], dtype=uint8)

In [31]:
img = mpimg.imread('test6.jpg')
draw_img = np.copy(img)

In [32]:
draw_img.shape

(720, 1280, 3)

In [33]:
img = img.astype(np.float32)/255

In [35]:
ystart = 400
ystop = 656
scale = 1.5
img_tosearch = img[ystart:ystop,:,:]

In [37]:
img_tosearch.shape

(256, 1280, 3)

In [38]:
import numpy as np
import cv2
from skimage.feature import hog

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [39]:
ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')

In [41]:
if scale != 1:
    imshape = ctrans_tosearch.shape
    ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))

In [42]:
ctrans_tosearch.shape

(170, 853, 3)

In [43]:
ch1 = ctrans_tosearch[:,:,0]
ch2 = ctrans_tosearch[:,:,1]
ch3 = ctrans_tosearch[:,:,2]

In [54]:
ch1.shape

(170, 853)

In [49]:
orient = 9
pix_per_cell = 8
cell_per_block = 2
nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
nfeat_per_block = orient*cell_per_block**2

In [50]:
window = 64
nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
cells_per_step = 2  # Instead of overlap, define how many cells to step
nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
nysteps = (nyblocks - nblocks_per_window) // cells_per_step

In [55]:
from scipy.ndimage.measurements import label

In [60]:
a = np.array([[0,0,1,1,0,0],
              [0,0,0,0,0,0],
              [1,1,0,0,1,0],
              [0,0,0,1,0,0]])
label(a)

(array([[0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [2, 2, 0, 0, 3, 0],
        [0, 0, 0, 4, 0, 0]], dtype=int32), 4)